In [ ]:
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
import keras

In [ ]:
train_ds, val_ds, test_ds = tfds.load(
    'wider_face',
    split=['train', 'validation', 'test'],
    shuffle_files=True
)

In [ ]:
def preprocess(sample):
    img=tf.image.resize(sample['image'],(128,128))
    img=img/255.0
    box=sample['faces']['bbox']
    return img, box

train_ds=train_ds.map(preprocess).cache().batch(16).shuffle(100).prefetch(tf.data.AUTOTUNE)
val_ds=val_ds.map(preprocess).cache().batch(16)
test_ds=test_ds.map(preprocess).cache().batch(16)

In [ ]:
#từ 1 tới 5 theo mô hình vgg16, bỏ phần dưới từ pooling layer ở block 5 đến hết cấu trúc vgg16 gốc
#từ 6 đến 11 là phần layer thêm vào theo cấu trúc ssd300
input=keras.Input(shape=(300,300,3))
#1
conv1_1=keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu',padding='same')(input)
conv1_2=keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu',padding='same')(conv1_1)
pooling1=keras.layers.MaxPool2D(pool_size=(2,2),strides=2)(conv1_2)
#2
conv2_1=keras.layers.Conv2D(128,kernel_size=(3,3),activation='relu',padding='same')(pooling1)
conv2_2=keras.layers.Conv2D(128,kernel_size=(3,3),activation='relu',padding='same')(conv2_1)
pooling2=keras.layers.MaxPool2D(pool_size=(2,2),strides=2)(conv2_2)
#3
conv3_1=keras.layers.Conv2D(256,kernel_size=(3,3),activation='relu',padding='same')(pooling2)
conv3_2=keras.layers.Conv2D(256,kernel_size=(3,3),activation='relu',padding='same')(conv3_1)
conv3_3=keras.layers.Conv2D(256,kernel_size=(3,3),activation='relu',padding='same')(conv3_2)
pooling3=keras.layers.MaxPool2D(pool_size=(2,2),strides=2)(conv3_3)
#4
conv4_1=keras.layers.Conv2D(512,kernel_size=(3,3),activation='relu',padding='same')(pooling3)
conv4_2=keras.layers.Conv2D(512,kernel_size=(3,3),activation='relu',padding='same')(conv4_1)
conv4_3=keras.layers.Conv2D(512,kernel_size=(3,3),activation='relu',padding='same')(conv4_2)
conv4_3_l2=tf.nn.l2_normalize(conv4_3,axis=-1)#feature map 1 38x38
pooling4=keras.layers.MaxPool2D(pool_size=(2,2),strides=2)(conv4_3)
#5
conv5_1=keras.layers.Conv2D(512,kernel_size=(3,3),activation='relu',padding='same')(pooling4)
conv5_2=keras.layers.Conv2D(512,kernel_size=(3,3),activation='relu',padding='same')(conv5_1)
conv5_3=keras.layers.Conv2D(512,kernel_size=(3,3),activation='relu',padding='same')(conv5_2)
#6
conv6_1=keras.layers.Conv2D(1024,dilation_rate=6,kernel_size=(3,3),activation='relu',padding='same')(conv5_3)
#7
conv7_1=keras.layers.Conv2D(1024,kernel_size=(1,1),activation='relu',padding='same')(conv6_1)#feature map 2 19x19
#8
conv8_1=keras.layers.Conv2D(256,kernel_size=(1,1),activation='relu',padding='same')(conv7_1)
conv8_2=keras.layers.Conv2D(512,kernel_size=(3,3),activation='relu',padding='same',strides=2)(conv8_1)#feature map 3 10x10
#9
conv9_1=keras.layers.Conv2D(128,kernel_size=(1,1),activation='relu',padding='same')(conv8_2)
conv9_2=keras.layers.Conv2D(256,kernel_size=(3,3),activation='relu',padding='same',strides=2)(conv9_1)#feature map 4 5x5
#10
conv10_1=keras.layers.Conv2D(128,kernel_size=(1,1),activation='relu',padding='same')(conv9_2)
conv10_2=keras.layers.Conv2D(256,kernel_size=(3,3),activation='relu',padding='valid',strides=1)(conv10_1)#feature map 5 3x3
#11
conv11_1=keras.layers.Conv2D(128,kernel_size=(1,1),activation='relu',padding='valid')(conv10_2)
conv11_2=keras.layers.Conv2D(256,kernel_size=(3,3),activation='relu',padding='valid',strides=1)(conv11_1)#feature map 6 1x1

model=keras.Model(inputs=input,outputs=[conv4_3_l2, conv7_1, conv8_2, conv9_2, conv10_2, conv11_2])

In [14]:
n_classes=2
n_boxes=4

def map_cls_reshape(layer):
    shape=keras.backend.int_shape(layer)
    map_cls=keras.layers.Conv2D(n_boxes*n_classes,kernel_size=3,padding='same')(layer)
    return keras.layers.Reshape((-1,n_classes))(map_cls)

def map_box_reshape(layer):
    shape=keras.backend.int_shape(layer)
    map_box=keras.layers.Conv2D(n_boxes*4,kernel_size=3,padding='same')(layer)
    return keras.layers.Reshape((-1,4))(map_box)

map1_cls=map_cls_reshape(conv4_3_l2)
map1_box=map_box_reshape(conv4_3_l2)

map2_cls=map_cls_reshape(conv7_1)
map2_box=map_box_reshape(conv7_1)

map3_cls=map_cls_reshape(conv8_2)
map3_box=map_box_reshape(conv8_2)

map4_cls=map_cls_reshape(conv9_2)
map4_box=map_box_reshape(conv9_2)

map5_cls=map_cls_reshape(conv10_2)
map5_box=map_box_reshape(conv10_2)

map6_cls=map_cls_reshape(conv11_2)
map6_box=map_box_reshape(conv11_2)